<a href="https://colab.research.google.com/github/six-choi/crawler/blob/main/%EC%82%B0%EC%9E%AC%EC%A7%81%EC%97%85%EB%B3%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# 필요한 라이브러리 설치
!pip install selenium pandas
!apt-get update # Ubuntu 업데이트
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 43 n

In [12]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# WebDriver 설정
driver = webdriver.Chrome(options=chrome_options)

In [14]:
# 웹사이트 열기 (URL을 원하는 대상으로 변경하세요)
driver.get('https://www.kosha.or.kr/kosha/data/occupationalDisease.do?mode=list&&articleLimit=1000&article.offset=0')
time.sleep(10)  # 페이지 로딩 대기

# 데이터를 저장할 빈 리스트 초기화
data = []

# 페이지 내 모든 링크의 개수를 찾습니다.
links_count = len(driver.find_elements(By.CSS_SELECTOR, 'td.board-list-title > div > a'))

for i in range(links_count):
    # 매번 요소를 새로 찾습니다.
    link = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'td.board-list-title > div > a'))
    )
    link.click()
    # 페이지 로딩 대기
    time.sleep(1)

    text = driver.find_element(By.CSS_SELECTOR, '#jwxe_main_content > div.ko.board.view > div.Board-view > div > table > tbody').text
    pdf_links = driver.find_elements(By.CSS_SELECTOR, '#view-downbox > ul > li > a')
    pdf_link = pdf_links[0].get_attribute('href') if pdf_links else 'No PDF'

    data.append({'text': text, 'pdf_link': pdf_link})

    driver.back()
    WebDriverWait(driver, 10).until(EC.staleness_of(link))

driver.quit()

# Pandas DataFrame으로 데이터 저장
df = pd.DataFrame(data)

# DataFrame을 CSV 파일로 저장
df.to_csv('/content/scraped_data.csv', index=False)

print("스크래핑 완료 및 파일 저장")

스크래핑 완료 및 파일 저장


In [16]:
from google.colab import files

# Download the UTF-8 encoded file
files.download("/content/scraped_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>